In [ ]:
import concurrent.futures
import connectorx as cx
import pandas as pd
import numpy as np
import os
import gc
import duckdb
from datetime import datetime, timedelta
import glob
import multiprocessing 
from functools import partial
import time
import traceback

In [ ]:
import logging

# 设置日志
logger = logging.getLogger(__name__)
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)


In [ ]:
def calculate_minute_level_apb(trading_date):
    """计算分钟级APB因子"""
    try:
        # 尝试在函数内部导入pandas，避免全局导入
        import pandas as pd
        
        # 如果传入的是字符串，转换为datetime对象
        if isinstance(trading_date, str):
            trading_date = pd.to_datetime(trading_date)
        
        date_str = trading_date.strftime('%Y%m%d')
        
        order_pth = f"/data/cephfs/order/{date_str}.parquet"
        
        # 确保输出目录存在
        output_dir = "./factors/Bid_minute_level_APB"
        os.makedirs(output_dir, exist_ok=True)

        output_file = f"{output_dir}/{date_str}.parquet"
        
        if not os.path.exists(order_pth):
            # logger.info(f"订单数据文件不存在: {order_pth}")
            return
        
        # 如果该日期的文件已存在，跳过处理
        if os.path.exists(output_file):
            logger.info(f"日期 {date_str} 已处理，跳过")
            return

        conn = duckdb.connect(database=':memory:')

        # 1. 读取毫秒级订单簿数据并转换为分钟级
        query_order = f"""
            WITH buy_orders_with_time AS(
                SELECT 
                    security_code,
                    order_side, 
                    order_type,
                    order_details,
                    order_price,
                    order_price_adj,
                    order_volume,
                    order_time,
                    ('{trading_date}'::DATE + MAKE_TIME(
                        FLOOR(order_time / 10000000)::int,
                        (FLOOR(order_time / 100000) % 100)::int,
                        (FLOOR(order_time / 1000) % 100)::numeric + 
                        (order_time % 1000)::numeric / 1000)
                    )::TIMESTAMP AS formatted_time
                FROM '{order_pth}'
                WHERE order_side = 1
                    AND order_type = 'A'
                    AND order_details = 'L'
                    AND order_price > 0
                    AND order_volume > 0
                    AND order_time >= 93000000
                    AND order_time < 145700000
            )
            SELECT
                security_code,
                order_price,
                order_volume,
                order_time,
                DATE_TRUNC('minute', formatted_time) AS order_minute
            FROM buy_orders_with_time
            """

        # 将数据加载到临时表中
        conn.execute(f"CREATE TEMPORARY TABLE minute_order AS {query_order}")


        minute_apb_query = """
        SELECT 
            security_code,
            order_minute,
            SUM(order_volume * order_price) / SUM(order_volume) AS vwap,
            AVG(order_price) AS twap,
            SUM(order_volume) AS total_volume
        FROM minute_order
        GROUP BY security_code, order_minute
        """
        conn.execute(f"CREATE TEMPORARY TABLE minute_stats AS {minute_apb_query}")


        apb_query = """
        SELECT 
            '{date_str}' AS date,
            security_code,
            order_minute,
            vwap,
            twap,
            CASE 
                WHEN twap > 0 AND vwap > 0 THEN LN(twap / vwap)
                ELSE NULL
            END AS apb,
            (twap - vwap) / vwap AS apb_alt,
            total_volume
        FROM minute_stats
        WHERE total_volume > 0
        """.format(date_str=date_str)
        
        apb_df = conn.execute(apb_query).fetchdf()

       
        # 10. 保存分钟级数据
        result_df.to_parquet(output_file)
        
        logger.info(f"成功处理日期: {date_str}, 记录数: {len(result_df)}")
        return True
    except Exception as e:
        logger.error(f"处理日期 {trading_date} 时出错: {str(e)}")
        logger.error(traceback.format_exc())
        return False


In [ ]:
def worker_process(date):
    """工作进程函数"""
    try:
        result = calculate_minute_level_apb(date)
        return (date, result, None)
    except Exception as e:
        return (date, False, str(e))


def process_all_trading_days(trading_dates, num_processes=None):
    """使用多进程处理所有交易日的数据
    
    Args:
        trading_dates: 要处理的交易日列表
        num_processes: 进程数，默认为CPU核心数
    """
    if num_processes is None:
        num_processes = multiprocessing.cpu_count()
    
    # 确保输出目录存在
    output_dir = "./factors/Bid_minute_level_APB"
    os.makedirs(output_dir, exist_ok=True)
    
    # 过滤掉已经处理过的日期
    dates_to_process = []
    for date in trading_dates:
        # 如果是字符串格式，转换为datetime对象
        if isinstance(date, str):
            try:
                date_obj = datetime.strptime(date, '%Y-%m-%d')
            except ValueError:
                try:
                    date_obj = datetime.strptime(date, '%Y%m%d')
                except ValueError:
                    logger.error(f"无法解析日期格式: {date}")
                    continue
        else:
            date_obj = date
            
        date_str = date_obj.strftime('%Y%m%d')
        output_file = f"{output_dir}/{date_str}.parquet"
        
        if not os.path.exists(output_file):
            dates_to_process.append(date)
    
    total_dates = len(dates_to_process)
    if total_dates == 0:
        logger.info("所有日期已处理完毕，无需再次处理")
        return
    
    logger.info(f"开始处理 {total_dates} 个交易日，使用 {num_processes} 个进程")
    
    # 创建进程池
    pool = multiprocessing.Pool(processes=num_processes)
    
    # 提交所有任务
    results = []
    for date in dates_to_process:
        result = pool.apply_async(worker_process, (date,))
        results.append(result)
    
    # 关闭进程池，不再接受新任务
    pool.close()
    
    # 显示进度
    completed = 0
    while completed < total_dates:
        completed = sum(1 for r in results if r.ready())
        logger.info(f"进度: {completed}/{total_dates}")
        time.sleep(10)  # 每10秒更新一次进度
    
    # 等待所有进程完成
    pool.join()
    
    # 处理结果
    success_count = 0
    for result in results:
        date, success, error = result.get()
        if success:
            success_count += 1
        else:
            logger.error(f"处理日期 {date} 时出错: {error}")
    
    logger.info(f"处理完成: 成功 {success_count}/{total_dates}")


# 使用示例
if __name__ == "__main__":

    start_date = '2016-06-20'
    end_date = '2025-06-30'

    trading_dates = pd.date_range(start=start_date, end=end_date, freq='D')
    
    process_all_trading_days(trading_dates, num_processes=20)


2025-07-25 17:37:26,463 - __main__ - INFO - 开始处理 1745 个交易日，使用 20 个进程
2025-07-25 17:37:28,373 - __main__ - INFO - 进度: 6/1745


2025-07-25 17:37:38,375 - __main__ - INFO - 进度: 6/1745
2025-07-25 17:37:45,821 - __main__ - ERROR - 处理日期 2016-06-22 00:00:00 时出错: Catalog Error: Table with name minute_percentiles does not exist!
Did you mean "minute_order"?

LINE 9:         JOIN minute_percentiles p ON m.security_code = p.security_code...
                     ^
2025-07-25 17:37:45,905 - __main__ - ERROR - Traceback (most recent call last):
  File "/tmp/ipykernel_2847448/2952075908.py", line 82, in calculate_minute_level_apb
    conn.execute(minute_query)
duckdb.duckdb.CatalogException: Catalog Error: Table with name minute_percentiles does not exist!
Did you mean "minute_order"?

LINE 9:         JOIN minute_percentiles p ON m.security_code = p.security_code...
                     ^

2025-07-25 17:37:46,230 - __main__ - ERROR - 处理日期 2016-06-27 00:00:00 时出错: Catalog Error: Table with name minute_percentiles does not exist!
Did you mean "minute_order"?

LINE 9:         JOIN minute_percentiles p ON m.security_code = p.se

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

...

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

2025-07-25 17:47:28,533 - __main__ - INFO - 进度: 1722/1745


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

2025-07-25 17:47:38,535 - __main__ - INFO - 进度: 1725/1745
2025-07-25 17:47:48,537 - __main__ - INFO - 进度: 1725/1745
2025-07-25 17:47:54,612 - __main__ - ERROR - 处理日期 2025-06-03 00:00:00 时出错: Catalog Error: Table with name minute_percentiles does not exist!
Did you mean "minute_order"?

LINE 9:         JOIN minute_percentiles p ON m.security_code = p.security_code...
                     ^
2025-07-25 17:47:55,027 - __main__ - ERROR - Traceback (most recent call last):
  File "/tmp/ipykernel_2847448/2952075908.py", line 82, in calculate_minute_level_apb
    conn.execute(minute_query)
duckdb.duckdb.CatalogException: Catalog Error: Table with name minute_percentiles does not exist!
Did you mean "minute_order"?

LINE 9:         JOIN minute_percentiles p ON m.security_code = p.security_code...
                     ^

2025-07-25 17:47:58,539 - __main__ - INFO - 进度: 1725/1745
2025-07-25 17:47:59,367 - __main__ - ERROR - 处理日期 2025-06-06 00:00:00 时出错: Catalog Error: Table with name minute_percen

In [ ]:
def calculate_daily_rolling_average(min_window=5):
    import glob
    import os
    import pandas as pd
    import concurrent.futures
    from functools import partial
    
    # 源数据目录
    data_dir = "./factors/Bid_minute_level_APB"
    # 日度统计结果保存目录
    daily_stats_dir = "./factors/Bid_minute_level_APB/DailyMinute"
    os.makedirs(daily_stats_dir, exist_ok=True)
    
    # 获取所有日期的parquet文件
    parquet_files = sorted(glob.glob(os.path.join(data_dir, "*.parquet")))
    
    if not parquet_files:
        raise ValueError("未找到分钟级数据文件")
    
    print(f"在目录 {data_dir} 中找到 {len(parquet_files)} 个Parquet文件")
    
    # 定义处理单个日期文件的函数
    def process_daily_file(file_path):
        try:
            # 读取单日数据
            df = pd.read_parquet(file_path)
            
            # 提取日期信息（从文件名）
            date_str = os.path.basename(file_path).split('.')[0]
            
            # 计算每个股票的平均值和标准差
            daily_value = df.groupby(['security_code'])['apb'].mean().reset_index()
            daily_std = df.groupby(['security_code'])['apb'].std().reset_index()
            
            # 合并平均值和标准差
            daily_factor = pd.merge(
                daily_value, 
                daily_std,
                on='security_code',
                how='inner',
                suffixes=('', '_std')
            )
            
            # 添加日期列
            daily_factor['date'] = date_str
            
            # 保存为新的parquet文件
            output_file = os.path.join(daily_stats_dir, f"{date_str}.parquet")
            daily_factor.to_parquet(output_file)
            
            print(f"处理完成: {date_str}, 包含 {len(daily_factor)} 条记录")
            return output_file
        except Exception as e:
            print(f"处理文件 {file_path} 时出错: {str(e)}")
            return None
    
    # 第一阶段：并行处理每个日期文件
    with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
        processed_files = list(executor.map(process_daily_file, parquet_files))
    
    # 过滤掉处理失败的文件
    processed_files = [f for f in processed_files if f is not None]
    print(f"成功处理 {len(processed_files)} 个日期文件")
    
    # 第二阶段：读取所有日度统计文件并合并
    daily_stats_files = sorted(glob.glob(os.path.join(daily_stats_dir, "*.parquet")))
    
    if not daily_stats_files:
        raise ValueError("未找到日度统计文件")
    
    # 读取并合并所有日度统计文件
    all_daily_stats = []
    for file in daily_stats_files:
        df = pd.read_parquet(file)
        all_daily_stats.append(df)
    
    # 合并所有日期的数据
    all_stats = pd.concat(all_daily_stats, ignore_index=True)
    
    # 确保日期格式正确并排序
    all_stats['date'] = pd.to_datetime(all_stats['date'], format='%Y%m%d')
    all_stats = all_stats.sort_values(['date', 'security_code'])
    
    # 获取所有个股代码
    all_securities = all_stats['security_code'].unique()
    print(f"共有 {len(all_securities)} 个不同的证券代码")
    
    # 最终输出目录
    output_path = "/data/home/lexuanchen/Factors/Order/Signal/Alltime_Improved_APB_2"
    os.makedirs(output_path, exist_ok=True)
    
    factor_names = ['apb', 'apb_std']
    
    # 定义处理每个因子的函数
    def process_factor(factor_name, all_stats, min_window):
        # 转置成宽表进行滚动计算
        pivot_data = all_stats.pivot(index='date', columns='security_code', values=factor_name)
        
        # 对宽表直接应用rolling
        rolling_avg = pivot_data.rolling(window=20, min_periods=min_window).mean()
        
        # 将结果转换回长格式
        factor_df = rolling_avg.stack().reset_index()
        factor_df.columns = ['date', 'security_code', f'{factor_name}_rolling']
        
        # 计算每日因子覆盖率
        non_nan_counts = factor_df.dropna(subset=[f'{factor_name}_rolling']).groupby('date').size()
        
        # 计算覆盖率
        coverage = non_nan_counts / len(all_securities)
        print(f"\n{factor_name}因子覆盖率统计: 平均={coverage.mean():.2f}, 最小={coverage.min():.2f}")
        
        # 恢复日期格式为字符串（如果需要）
        factor_df['date'] = factor_df['date'].dt.strftime('%Y%m%d')
        
        # 保存结果
        output_file_path = f"{output_path}/M_M_Order_Bid_20d_daily_{factor_name}.csv"
        factor_df.to_csv(output_file_path, index=False)
        
        return output_file_path
    
    # 使用多线程处理每个因子
    with concurrent.futures.ThreadPoolExecutor(max_workers=len(factor_names)) as executor:
        process_func = partial(process_factor, all_stats=all_stats, min_window=min_window)
        result_files = list(executor.map(process_func, factor_names))
    
    print(f"所有因子处理完成，结果保存在: {output_path}")
    return output_path


In [ ]:
output_path2 = calculate_daily_rolling_average(min_window=5)

ValueError: 未找到分钟级数据文件